In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import gillespy2
from gillespy2 import Model, Species, Parameter, Reaction, Event, \
                      EventTrigger, EventAssignment, RateRule, \
                      AssignmentRule, FunctionDefinition
from gillespy2 import SSACSolver
# from gillespy2 import TauLeapingCSolver
# from gillespy2 import ODECSolver
# from gillespy2 import TauHybridSolver

# Vilar_Oscillator

In [3]:
class VilarOscillator(Model):
    def __init__(self, parameter_values=None):
        Model.__init__(self, name="Vilar_Oscillator")
        self.volume = 1

        # Parameters
        self.add_parameter(Parameter(name="alphaA", expression="50"))
        self.add_parameter(Parameter(name="alphaA_prime", expression="500"))
        self.add_parameter(Parameter(name="alphaR", expression="0.01"))
        self.add_parameter(Parameter(name="alphaR_prime", expression="50"))
        self.add_parameter(Parameter(name="betaA", expression="50"))
        self.add_parameter(Parameter(name="betaR", expression="5"))
        self.add_parameter(Parameter(name="deltaMA", expression="10"))
        self.add_parameter(Parameter(name="deltaMR", expression="0.5"))
        self.add_parameter(Parameter(name="deltaA", expression="1"))
        self.add_parameter(Parameter(name="deltaR", expression="0.2"))
        self.add_parameter(Parameter(name="gammaA", expression="1"))
        self.add_parameter(Parameter(name="gammaR", expression="1"))
        self.add_parameter(Parameter(name="gammaC", expression="2"))
        self.add_parameter(Parameter(name="thetaA", expression="50"))
        self.add_parameter(Parameter(name="thetaR", expression="100"))

        # Variables
        self.add_species(Species(name="Da", initial_value=1, mode="discrete"))
        self.add_species(Species(name="Da_prime", initial_value=0, mode="discrete"))
        self.add_species(Species(name="Ma", initial_value=0, mode="discrete"))
        self.add_species(Species(name="Dr", initial_value=1, mode="discrete"))
        self.add_species(Species(name="Dr_prime", initial_value=0, mode="discrete"))
        self.add_species(Species(name="Mr", initial_value=0, mode="discrete"))
        self.add_species(Species(name="C", initial_value=0, mode="discrete"))
        self.add_species(Species(name="A", initial_value=0, mode="discrete"))
        self.add_species(Species(name="R", initial_value=0, mode="discrete"))

        # Reactions
        self.add_reaction(Reaction(name="r1", reactants={'A': 1, 'R': 1}, products={'C': 1}, rate="gammaC"))
        self.add_reaction(Reaction(name="r2", reactants={'A': 1}, products={}, rate="deltaA"))
        self.add_reaction(Reaction(name="r3", reactants={'C': 1}, products={'R': 1}, rate="deltaA"))
        self.add_reaction(Reaction(name="r4", reactants={'R': 1}, products={}, rate="deltaR"))
        self.add_reaction(Reaction(name="r5", reactants={'A': 1, 'Da': 1}, products={'Da_prime': 1}, rate="gammaA"))
        self.add_reaction(Reaction(name="r6", reactants={'Da_prime': 1}, products={'A': 1, 'Da': 1}, rate="thetaA"))
        self.add_reaction(Reaction(name="r7", reactants={'Da': 1}, products={'Da': 1, 'Ma': 1}, rate="alphaA"))
        self.add_reaction(Reaction(name="r8", reactants={'Da_prime': 1}, products={'Da_prime': 1, 'Ma': 1}, rate="alphaA_prime"))
        self.add_reaction(Reaction(name="r9", reactants={'Ma': 1}, products={}, rate="deltaMA"))
        self.add_reaction(Reaction(name="r10", reactants={'Ma': 1}, products={'A': 1, 'Ma': 1}, rate="betaA"))
        self.add_reaction(Reaction(name="r11", reactants={'A': 1, 'Dr': 1}, products={'Dr_prime': 1}, rate="gammaR"))
        self.add_reaction(Reaction(name="r12", reactants={'Dr_prime': 1}, products={'A': 1, 'Dr': 1}, rate="thetaR"))
        self.add_reaction(Reaction(name="r13", reactants={'Dr': 1}, products={'Dr': 1, 'Mr': 1}, rate="alphaR"))
        self.add_reaction(Reaction(name="r14", reactants={'Dr_prime': 1}, products={'Dr_prime': 1, 'Mr': 1}, rate="alphaR_prime"))
        self.add_reaction(Reaction(name="r15", reactants={'Mr': 1}, products={}, rate="deltaMR"))
        self.add_reaction(Reaction(name="r16", reactants={'Mr': 1}, products={'Mr': 1, 'R': 1}, rate="betaR"))

        # Timespan
        self.timespan(np.arange(0, 201, 1))

In [4]:
model = VilarOscillator()

# Simulation Parameters

In [5]:
def configure_simulation():
    solver = SSACSolver(model=model)
    kwargs = {
        "solver":solver,
        "number_of_trajectories":100,
        # "seed":None,
        # "tau_tol":0.03,
        # "integrator_options":{'rtol': 0.001, 'atol': 1e-06},
    }
    return kwargs

# Model Inference

In [6]:
from tsfresh.feature_extraction.settings import MinimalFCParameters

## Generate some fixed(observed) data based on default parameters of the model

In [7]:
kwargs = configure_simulation()
fixed_data = model.run(**kwargs)

In [8]:
# Reshape the data and remove timepoints array
fixed_data = fixed_data.to_array()
fixed_data = np.asarray([x.T for x in fixed_data])
fixed_data = fixed_data[:, 1:, :]

## Define prior distribution

In [9]:
from sciope.utilities.priors import uniform_prior

# take default from mode 1 as reference
default_param = np.array(list(model.listOfParameters.items()))[:, 1]

bound = []
for exp in default_param:
    bound.append(float(exp.expression))

# Set the bounds
bound = np.array(bound)
dmin = bound * 0.1
dmax = bound * 2.0

# Here we use uniform prior
uni_prior = uniform_prior.UniformPrior(dmin, dmax)

## Define simulator

In [10]:
def get_variables(params, model):
    # params - array, need to have the same order as model.listOfParameters
    variables = {}
    for e, pname in enumerate(model.listOfParameters.keys()):
        variables[pname] = params[e]
    return variables

# Here we use the GillesPy2 Solver
def simulator(params, model):
    variables = get_variables(params, model)

    res = model.run(**kwargs, variables=variables)
    res = res.to_array()
    tot_res = np.asarray([x.T for x in res]) # reshape to (N, S, T)
    # should not contain timepoints
    tot_res = tot_res[:, 1:, :]

    return tot_res

# Wrapper, simulator function to abc should should only take one argument (the parameter point)
def simulator2(x):
    return simulator(x, model=model)

## Define summary statistics and distance function

In [11]:
from sciope.utilities.summarystats import auto_tsfresh
from sciope.utilities.distancefunctions import naive_squared

# Function to generate summary statistics
summ_func = auto_tsfresh.SummariesTSFRESH()

# Distance
ns = naive_squared.NaiveSquaredDistance()

## Start local cluster using dask client

In [12]:
from dask.distributed import Client

c = Client()

## Start abc instance

In [13]:
from sciope.inference.abc_inference import ABC

abc = ABC(fixed_data, sim=simulator2, prior_function=uni_prior, summaries_function=summ_func.compute, distance_function=ns)

In [14]:
# First compute the fixed(observed) mean
abc.compute_fixed_mean(chunk_size=2)

In [ ]:
res = abc.infer(num_samples=100, batch_size=10, chunk_size=2)

/opt/conda/lib/python3.8/site-packages/distributed/worker.py:3801: UserWarning: Large object of size 2.97 MiB detected in task graph: 
  (<function ABC.rejection_sampling.<locals>.<lambda ... -2ea6f00228d8')
Consider scattering large objects ahead of time
with client.scatter to reduce scheduler burden and 
keep data on workers

    future = client.submit(func, big_data)    # bad

    big_future = client.scatter(big_data)     # good
    future = client.submit(func, big_future)  # good
  warnings.warn(


In [ ]:
from sklearn.metrics import mean_absolute_error

mae_inference = mean_absolute_error(bound, abc.results['inferred_parameters'])